# Homework 3 - Interactive Viz

In [2]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import folium
import re
import sys

## European unemployment rates

**Go to the [eurostat](http://ec.europa.eu/eurostat/data/database) website and try to find a dataset that includes the european unemployment rates at a recent date.**

   **Use this data to build a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) which shows the unemployment rate in Europe at a country level. Think about [the colors you use](https://carto.com/academy/courses/intermediate-design/choose-colors-1/), how you decided to [split the intervals into data classes](http://gisgeography.com/choropleth-maps-data-classification/) or which interactions you could add in order to make the visualization intuitive and expressive. Compare Switzerland's unemployment rate to that of the rest of Europe.**

On the eurostat website, we found our dataset under 'Database by themes/Population and social conditions/Labour Market(labour)/Employment and unemployment/LFS main indicators/Unemployment - LFS adjusted series/Unemployment by sex and age - annual average'. It provides the unemployment rates up to 2016 for all european countries. On the contrary of its name, both sex and all ages are mixed.

In [3]:
CSV_PATH = 'data/question1/une_rt_a_1_Data.csv'

europe_df = pd.read_csv(CSV_PATH) 

europe_df = europe_df.loc[europe_df['TIME'] == 2016]
europe_df = europe_df.loc[europe_df['UNIT'] == 'Percentage of active population']
europe_df = europe_df.drop(['TIME','AGE', 'SEX','UNIT','Flag and Footnotes'], axis = 1)
europe_df = europe_df[6:]
europe_df = europe_df[:30]
# Clean Germany's name
europe_df['GEO'] = europe_df['GEO'].str.replace(r"\(.*\)","")
europe_df['Value'] = europe_df['Value'].astype(float)
europe_df = europe_df[~(europe_df['GEO'] == 'Cyprus')]
europe_df = europe_df[~(europe_df['GEO'] == 'Malta')]

In [4]:
for line in europe_df['GEO']:
    if line == 'Germany ':
        line = 'Germany'
        print(line)
europe_df.head()

Germany


,GEO,Value
1073,Belgium,7.8
1076,Bulgaria,7.6
1079,Czech Republic,4.0
1082,Denmark,6.2
1085,Germany,4.1


In [5]:
state_topo_path = r'topojson/custom.geo.json'
topo_json_data = json.load(open(state_topo_path))

In [6]:
state_geo = r'topojson/custom.geo.json'
topo_json_data = json.load(open(state_geo))

us_map = folium.Map(location=[54,17], zoom_start=4)
us_map.choropleth(geo_data=topo_json_data, data=europe_df,
             columns=['GEO', 'Value'],
             key_on='feature.properties.name_long',
             fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Percentage of Unemployment (%)')
us_map

In [7]:
swiss_unemploy_2016 = 3.5

## Swiss Cantons unemployement

**Go to the [amstat](https://www.amstat.ch) website to find a dataset that includes the unemployment rates in Switzerland at a recent date.**

   > *HINT* Go to the `details` tab to find the raw data you need. If you do not speak French, German or Italian, think of using free translation services to navigate your way through. 

  ** Use this data to build another Choropleth map, this time showing the unemployment rate at the level of swiss cantons. Again, try to make the map as expressive as possible, and comment on the trends you observe.**

  ** The Swiss Confederation defines the rates you have just plotted as the number of people looking for a job divided by the size of the active population (scaled by 100). This is surely a valid choice, but as we discussed one could argue for a different categorization.**

  ** Copy the map you have just created, but this time don't count in your statistics people who already have a job and are looking for a new one. How do your observations change ? You can repeat this with different choices of categories to see how selecting different metrics can lead to different interpretations of the same data.**

In [26]:
import csv
TXT_PATH = 'data/question2/swiss_unemployment.txt'

swiss_df = pd.read_table(TXT_PATH, ',') 

In [27]:
swiss_df

,Canton,Mois,Janvier 2017,Janvier 2017.1,Janvier 2017.2,Janvier 2017.3,Janvier 2017.4,Janvier 2017.5,Janvier 2017.6,Janvier 2017.7,...,Août 2017.6,Août 2017.7,Septembre 2017,Septembre 2017.1,Septembre 2017.2,Septembre 2017.3,Septembre 2017.4,Septembre 2017.5,Septembre 2017.6,Septembre 2017.7
0,,Mesures,Chômeurs inscrits,Entrées au chômage,Sortis du chômage,Chômeurs jeunes,Chômeurs de longue durée,Demandeurs d'emploi,Demandeurs d'emploi entrés,Demandeurs d'emploi sortis,...,Demandeurs d'emploi entrés,Demandeurs d'emploi sortis,Chômeurs inscrits,Entrées au chômage,Sortis du chômage,Chômeurs jeunes,Chômeurs de longue durée,Demandeurs d'emploi,Demandeurs d'emploi entrés,Demandeurs d'emploi sortis
1,Zurich,NaN,32'387,5'922,5'108,3'800,5'429,39'340,5'896,5'157,...,4'827,5'385,27'225,5'067,5'340,3'420,4'753,34'156,5'071,5'438
2,Berne,NaN,16'954,3'201,2'873,2'324,2'744,21'586,2'694,2'732,...,2'617,2'675,13'658,2'627,2'797,2'003,2'336,18'385,2'590,2'596
3,Lucerne,NaN,4'985,1'393,1'299,812,586,7'950,1'147,1'076,...,1'048,988,3'885,1'166,1'275,664,453,6'756,945,1'043
4,Uri,NaN,297,151,96,41,22,464,116,59,...,54,52,112,43,54,15,13,257,52,64
5,Schwyz,NaN,1'794,487,379,196,205,2'581,398,331,...,356,346,1'455,345,363,185,189,2'229,346,362
6,Obwald,NaN,217,102,107,33,19,378,83,64,...,59,48,153,57,69,21,10,319,49,53
7,Nidwald,NaN,305,140,139,36,24,535,98,75,...,77,86,248,99,98,31,24,436,77,66
8,Glaris,NaN,567,106,107,85,78,876,103,81,...,98,73,416,91,109,60,78,713,78,95
9,Zoug,NaN,1'756,450,383,172,259,2'831,357,335,...,305,348,1'543,331,392,133,248,2'615,326,374


## Foreign workers in Switzerland


**Use the [amstat](https://www.amstat.ch) website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between *Swiss* and *foreign* workers.**

   **The Economic Secretary (SECO) releases [a monthly report](https://www.seco.admin.ch/seco/fr/home/Arbeit/Arbeitslosenversicherung/arbeitslosenzahlen.html) on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for *foreign* (`5.1%`) and *Swiss* (`2.2%`) workers. **

  ** Show the difference in unemployment rates between the two categories in each canton on a Choropleth map (*hint* The easy way is to show two separate maps, but can you think of something better ?). Where are the differences most visible ? Why do you think that is ?**

  ** Now let's refine the analysis by adding the differences between age groups. As you may have guessed it is nearly impossible to plot so many variables on a map. Make a bar plot, which is a better suited visualization tool for this type of multivariate data.**

## Röstigraben

** *BONUS*: using the map you have just built, and the geographical information contained in it, could you give a *rough estimate* of the difference in unemployment rates between the areas divided by the [Röstigraben](https://en.wikipedia.org/wiki/R%C3%B6stigraben)?**